In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#loading Data
import glob

video_files =  glob.glob('/content/drive/My Drive/DeepFake/original_sequences/youtube/c23/videos/*.mp4')
video_files +=  glob.glob('/content/drive/My Drive/DeepFake/original_sequences/actors/c23/videos/*.mp4')

#print('ReaL:', video_files, '\n')

fake_vids = glob.glob('/content/drive/My Drive/DeepFake/manipulated_sequences/NeuralTextures/c23/videos/*.mp4')
fake_vids += glob.glob('/content/drive/My Drive/DeepFake/manipulated_sequences/FaceSwap/c23/videos/*.mp4')
fake_vids += glob.glob('/content/drive/My Drive/DeepFake/manipulated_sequences/FaceShifter/c23/videos/*.mp4')
fake_vids += glob.glob('/content/drive/My Drive/DeepFake/manipulated_sequences/Face2Face/c23/videos/*.mp4')
fake_vids += glob.glob('/content/drive/My Drive/DeepFake/manipulated_sequences/Deepfakes/c23/videos/*.mp4')
fake_vids += glob.glob('/content/drive/My Drive/DeepFake/manipulated_sequences/DeepFakeDetection/c23/videos/*.mp4')

#print('Fake:' , fake_vids, '\n')

Real Videos : 169 Fake Videos : 626


In [ ]:
#extracting frames
import cv2
def getFrames(vid_path):
  vidObj = cv2.VideoCapture(vid_path)
  cont = 1
  while cont:
    cont, image = vidObj.read()
    if cont == 1:
      yield image


In [ ]:
!mkdir '/content/drive/My Drive/Real_Face_videos_after_cropping'
!mkdir '/content/drive/My Drive/Fake_Face_videos_after_cropping'
!mkdir '/content/drive/My Drive/CSVFile'

In [ ]:
!pip install dlib
import dlib
!pip3 install face_recognition
import face_recognition

     |████████████████████████████████| 100.2MB 74kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566184 sha256=504dc591e403d615ade67867b3a18c2eb0bc4b9fd15632c964347fc03eb3e4b9
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [ ]:
import os
import cv2
import csv
from tqdm.autonotebook import tqdm

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  after removing the cwd from sys.path.


In [ ]:
def createVideosFace(videos,outputPath):
  for each in tqdm(videos):
    vid_path = os.path.join(outputPath,each.split('/')[-1])
    exists = glob.glob(vid_path)
    if len(exists)!= 0:
      vid_path = os.path.join(outputPath,each.split('/')[-4]+each.split('/')[-1])
    videoFrames = []
    booll = 0
    Faces = []
    vidFrames2 = []
    out = cv2.VideoWriter(vid_path,cv2.VideoWriter_fourcc('M','J','P','G'), 30, (112,112))
    for i,frame in enumerate(getFrames(each)):
      if i<=150:
        videoFrames.append(frame)
        face_locations = face_recognition.face_locations(frame, model="cnn")
        for i,face in enumerate(face_locations):
            if(len(face) != 0): 
              top,right,bottom,left = face         
            try:
              out.write(cv2.resize(videoFrames[i][top:bottom,left:right,:],(112,112)))
            except:
              pass
    videoFrames = []
    try:
      del top,right,bottom,left
    except:
      pass
    out.release()

In [ ]:
createVideosFace(video_files,'/content/drive/My Drive/Real_Face_videos_after_cropping')
createVideosFace(fake_vids,'/content/drive/My Drive/Fake_Face_videos_after_cropping')

In [ ]:
totalVids = video_files + fake_vids
len(totalVids)

80

In [ ]:
import random


vids =  glob.glob('/content/drive/My Drive/Real_Face_videos_after_cropping/*.mp4')
vids1 =  glob.glob('/content/drive/My Drive/Fake_Face_videos_after_cropping/*.mp4')

csvRows = [[]]
csvRows.append(["File Name", "Label"])
for each in vids:
  csvRows.append([each.split('/')[-1],'Real'])
for each in vids1:
  csvRows.append([each.split('/')[-1],'Fake'])

with open('/content/drive/My Drive/CSVFile/videofileslabels.csv', 'w') as file:
  writer = csv.writer(file)
  writer.writerows(csvRows)
   
vids += vids1
count =0
print(len(vids))
for i in vids:
    count+=1
    vid = cv2.VideoCapture(i)
    try:
      if not vid.isOpened():   #corrupted video
        vids.remove(i)
        raise NameError("Phew!! Video is Corrupted !!")
    except:
      pass
    else:
      pass
      #print('Valid Video !!!')

print(len(vids))

#shuffle the dataset to avoid training on same data

random.shuffle(vids)
random.shuffle(vids)
for video_file in vids:
  cap = cv2.VideoCapture(video_file)
  if(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))<120): #removing videos less than 120fps(4sec)
    vids.remove(video_file)
    continue


86
85


In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import torch
import torchvision
from torchvision import transforms

class MakeVideosDataset(Dataset):
  def __init__(self,videonames,labels,frameslength = 40,transform = None):
    self.videonames = videonames
    self.labels = labels
    self.count = frameslength
    self.trans = transform
  def __len__(self):
    return len(self.videonames)
    
  def __getitem__(self,index):
    video = self.videonames[index]
    label = self.labels[index]
    frames=[]
    filename = video.split('/')[-1]

    label = self.labels[index]
    if(label == 'Fake'):
      label = 0
    if(label == 'Real'):
      label = 1
    for i,frame in enumerate(getFrames(video)):
      frames.append(self.trans(frame))
      if(len(frames) == self.count):
        break
    frames = torch.stack(frames)
    frames = frames[:self.count]
  
    return frames,label

def im_plot(tensor):
  image = tensor.cpu().numpy().transpose(1,2,0)
  b,g,r = cv2.split(image)
  image = cv2.merge((r,g,b))
  image = image*[0.22803, 0.22145, 0.216989] +  [0.43216, 0.394666, 0.37645]
  image = image*255.0
  plt.imshow(image.astype(int))
  plt.show()




In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision import transforms

df = pd.read_csv('/content/drive/My Drive/CSVFile/videofileslabels.csv')
filenames = df['File Name']
labels = df['Label']
#print(labels)
X_train = vids[:int(0.8*len(vids))]
Y_test = vids[int(0.8*len(vids)):]

im_size = 112
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_transforms = transforms.Compose([
                                        transforms.ToPILImage(),
                                        transforms.Resize((im_size,im_size)),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean,std)])

data = MakeVideosDataset(X_train,labels,frameslength = 10,transform = train_transforms)
vdata = MakeVideosDataset(Y_test,labels,frameslength = 10,transform = train_transforms)

t_loader = DataLoader(dataset = data,batch_size = 4,shuffle = True,num_workers = 4)
v_loader = DataLoader(dataset = vdata,batch_size = 4,shuffle = True,num_workers = 4)

#image,label = data[21]
#im_plot(image[1,:,:,:])
data[0][0].shape



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


torch.Size([10, 3, 112, 112])

In [ ]:
from torch import nn
from torchvision import models

class Model(nn.Module):
    def __init__(self, num_classes,latent_dim= 2048, lstm_layers=1 , hidden_dim = 2048, bidirectional = False):
        super(Model, self).__init__()
        model = models.resnext50_32x4d(pretrained = True) #Residual Network CNN
        self.model = nn.Sequential(*list(model.children())[:-2])
        self.lstm = nn.LSTM(latent_dim,hidden_dim, lstm_layers,  bidirectional)
        self.relu = nn.LeakyReLU()
        self.dp = nn.Dropout(0.4)
        self.linear1 = nn.Linear(2048,num_classes)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
    def forward(self, x):
        batch_size,seq_length, c, h, w = x.shape
        x = x.view(batch_size * seq_length, c, h, w)
        fmap = self.model(x)
        x = self.avgpool(fmap)
        x = x.view(batch_size,seq_length,2048)
        x_lstm,_ = self.lstm(x,None)
        return fmap,self.dp(self.linear1(torch.mean(x_lstm,dim = 1)))
        #return x_lstm

In [ ]:
import numpy as np
model = Model(2).cuda()
a = model(torch.from_numpy(np.empty((1,20,3,112,112))).type(torch.cuda.FloatTensor))
print(a)

(tensor([[[[9.9543e-02, 4.4869e-02, 9.1683e-01, 2.0721e-01],
          [0.0000e+00, 8.9498e-01, 1.2581e+00, 6.4056e-01],
          [0.0000e+00, 5.0756e-01, 1.3630e-01, 1.2502e+00],
          [0.0000e+00, 4.3099e-01, 2.6047e-02, 1.4396e+00]],

         [[0.0000e+00, 6.3883e-01, 9.9331e-01, 4.7795e-01],
          [0.0000e+00, 7.8276e-01, 7.9952e-01, 9.3080e-01],
          [0.0000e+00, 6.4783e-01, 1.1627e+00, 1.3978e+00],
          [0.0000e+00, 4.0856e-01, 1.0398e-01, 3.4952e-01]],

         [[3.7643e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [7.4328e-01, 6.6736e-01, 9.4043e-01, 0.0000e+00],
          [1.3036e+00, 6.4928e-03, 5.4710e-01, 0.0000e+00],
          [1.0907e+00, 1.1043e+00, 1.4767e+00, 9.3433e-01]],

         ...,

         [[0.0000e+00, 0.0000e+00, 1.5648e-01, 0.0000e+00],
          [0.0000e+00, 5.7000e-01, 6.7901e-01, 0.0000e+00],
          [4.8722e-01, 6.0042e-01, 1.2955e+00, 2.2123e-01],
          [2.1556e-01, 1.0206e+00, 5.4502e-01, 5.4633e-01]],

         [[0.000

In [ ]:
class Average(object):
    def __init__(self):
        self.reset()
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
def calculate_accuracy(outputs, targets):
    batch_size = targets.size(0)

    _, pred = outputs.topk(1, 1, True)
    pred = pred.t()
    correct = pred.eq(targets.view(1, -1))
    n_correct_elems = correct.float().sum().item()
    return 100* n_correct_elems / batch_size

In [ ]:
import sys
def train(epoch, num_epochs, data_loader, model, criterion, optimizer):
    model.train()
    losses = Average()
    accuracies = Average()
    t = []
    for i, (inputs, targets) in enumerate(data_loader):
        if torch.cuda.is_available():
            targets = targets.type(torch.cuda.LongTensor)
            inputs = inputs.cuda()
        _,outputs = model(inputs)
        loss  = criterion(outputs,targets.type(torch.cuda.LongTensor))
        acc = calculate_accuracy(outputs, targets.type(torch.cuda.LongTensor))
        losses.update(loss.item(), inputs.size(0))
        accuracies.update(acc, inputs.size(0))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        sys.stdout.write(
                "\r[Epoch %d/%d] [Batch %d / %d] [Loss: %f, Acc: %.2f%%]"
                % (
                    epoch,
                    num_epochs,
                    i,
                    len(data_loader),
                    losses.avg,
                    accuracies.avg))
    torch.save(model.state_dict(),'/content/checkpoint.pt')
    return losses.avg,accuracies.avg


optimizer = torch.optim.Adam(model.parameters(), lr= 1e-5 ,weight_decay = 1e-5)
criterion = nn.CrossEntropyLoss().cuda()

trainingloss = []
trainingaccuracy = []
for epoch in range(20):
    l, acc = train(epoch,20,t_loader,model,criterion,optimizer)
    trainingloss.append(l)
    trainingaccuracy.append(acc)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[Epoch 19/20] [Batch 16 / 17] [Loss: 0.314350, Acc: 91.18%]

In [ ]:
def test(epoch,model, data_loader ,criterion):
    print('Testing')
    model.eval()
    losses = Average()
    accuracies = Average()
    pred = []
    true = []
    count = 0
    with torch.no_grad():
        for i, (inputs, targets) in enumerate(data_loader):
            if torch.cuda.is_available():
                targets = targets.cuda().type(torch.cuda.FloatTensor)
                inputs = inputs.cuda()
            _,outputs = model(inputs)
            loss = torch.mean(criterion(outputs, targets.type(torch.cuda.LongTensor)))
            acc = calculate_accuracy(outputs,targets.type(torch.cuda.LongTensor))
            _,p = torch.max(outputs,1) 
            true += (targets.type(torch.cuda.LongTensor)).detach().cpu().numpy().reshape(len(targets)).tolist()
            pred += p.detach().cpu().numpy().reshape(len(p)).tolist()
            losses.update(loss.item(), inputs.size(0))
            accuracies.update(acc, inputs.size(0))
            sys.stdout.write(
                    "\r[Batch %d / %d]  [Loss: %f, Acc: %.2f%%]"
                    % (
                        i,
                        len(data_loader),
                        losses.avg,
                        accuracies.avg
                        )
                    )
        print('\nAccuracy {}'.format(accuracies.avg))
    return true,pred,losses.avg,accuracies.avg

tesingloss = []
testingacc = []
for epoch in range(20):
    true,pred,tl,t_acc = test(epoch,model,v_loader,criterion)
    tesingloss.append(tl)
    testingacc.append(t_acc)

Testing


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[Batch 4 / 5]  [Loss: 1.875634, Acc: 29.41%]
Accuracy 29.41176470588235
Testing
[Batch 4 / 5]  [Loss: 1.835700, Acc: 29.41%]
Accuracy 29.41176470588235
Testing
[Batch 4 / 5]  [Loss: 1.813352, Acc: 29.41%]
Accuracy 29.41176470588235
Testing
[Batch 4 / 5]  [Loss: 1.814552, Acc: 29.41%]
Accuracy 29.41176470588235
Testing
[Batch 4 / 5]  [Loss: 1.868422, Acc: 29.41%]
Accuracy 29.41176470588235
Testing
[Batch 4 / 5]  [Loss: 1.848702, Acc: 29.41%]
Accuracy 29.41176470588235
Testing
[Batch 4 / 5]  [Loss: 1.875714, Acc: 29.41%]
Accuracy 29.41176470588235
Testing
[Batch 4 / 5]  [Loss: 1.871994, Acc: 29.41%]
Accuracy 29.41176470588235
Testing
[Batch 4 / 5]  [Loss: 1.859832, Acc: 29.41%]
Accuracy 29.41176470588235
Testing
[Batch 4 / 5]  [Loss: 1.871200, Acc: 29.41%]
Accuracy 29.41176470588235
Testing
[Batch 4 / 5]  [Loss: 1.862528, Acc: 29.41%]
Accuracy 29.41176470588235
Testing
[Batch 4 / 5]  [Loss: 1.845639, Acc: 29.41%]
Accuracy 29.41176470588235
Testing
[Batch 4 / 5]  [Loss: 1.840166, Acc: 29.

In [ ]:
from sklearn.metrics import confusion_matrix
def accuracy(confusion_matrix):
  diagnol = confusion_matrix.trace()
  sum = confusion_matrix.sum()
  return diagnol/sum


print(confusion_matrix(true,pred))
print("Accuracy: ", )

NameError: ignored